In [1]:
%pip install langchain-google-genai langchain-core pytz python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage # [수정] ToolMessage 추가
from langchain_core.tools import tool
from datetime import datetime
import pytz

load_dotenv()

True

In [29]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

response = llm.invoke([HumanMessage("잘 지냈어?")])
print(response.content)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 52.924529861s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 2
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 47.820324084s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 2
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 47
}
]

In [ ]:
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time

In [ ]:
# --- [원본 코드 스니펫 시작] ---
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)
# --- [원본 코드 스니펫 끝] ---

print("Gemini 모델이 get_current_time 도구와 성공적으로 바인딩되었습니다.")
print(llm_with_tools)

Gemini 모델이 get_current_time 도구와 성공적으로 바인딩되었습니다.
bound=ChatGoogleGenerativeAI(model='models/gemini-2.5-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000027DCE3EFBB0>, default_metadata=(), model_kwargs={}) kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_current_time', 'description': '현재 시각을 반환하는 함수', 'parameters': {'properties': {'timezone': {'description': "타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함", 'type': 'string'}, 'location': {'description': '지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨', 'type': 'string'}}, 'required': ['timezone', 'location'], 'type': 'object'}}}]} config={} config_factories=[]


In [ ]:
# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력 (AIMessage(tool_calls=...)가 출력됨)
print("--- 1차 호출 결과 (도구 호출 요청) ---")
print(response)
print("\n--- 누적된 messages 리스트 ---")
print(messages)
# --- [원본 코드 스니펫 끝] ---

--- 1차 호출 결과 (도구 호출 요청) ---
content='' additional_kwargs={'function_call': {'name': 'get_current_time', 'arguments': '{"location": "\\ubd80\\uc0b0", "timezone": "Asia/Seoul"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'} id='lc_run--0ebe6789-0ef1-41ab-afb2-ae13b11d04d6-0' tool_calls=[{'name': 'get_current_time', 'args': {'location': '부산', 'timezone': 'Asia/Seoul'}, 'id': '8f1d54a4-6915-4faa-90a6-e7a05df0f125', 'type': 'tool_call'}] usage_metadata={'input_tokens': 118, 'output_tokens': 219, 'total_tokens': 337, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 195}}

--- 누적된 messages 리스트 ---
[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMe

In [ ]:
# --- [오류가 발생한 코드 스니펫 (Cell 5) - 수정됨] ---
print("--- [도구 실행 단계] ---")
if response.tool_calls:
    for tool_call in response.tool_calls:
        selected_tool = tool_dict[tool_call["name"]]
        
        # --- [수정된 부분 시작] ---
        # 1. .invoke() 대신 **args로 함수를 직접 호출합니다.
        args = tool_call['args']
        tool_output = selected_tool(**args)
        
        # 2. 결과를 ToolMessage로 래핑합니다.
        tool_msg = ToolMessage(
            content=tool_output,
            tool_call_id=tool_call['id']
        )
        # --- [수정된 부분 끝] ---

        messages.append(tool_msg) # 도구 실행 결과를 messages에 추가
        print("도구 실행 완료. messages 리스트에 ToolMessage가 추가되었습니다.")
    print(messages[-1]) # 추가된 ToolMessage 확인
else:
    print("AI가 도구를 호출하지 않았습니다.")

--- [도구 실행 단계] ---
Asia/Seoul (부산) 현재시각 2025-10-27 18:58:21 
도구 실행 완료. messages 리스트에 ToolMessage가 추가되었습니다.
content='Asia/Seoul (부산) 현재시각 2025-10-27 18:58:21 ' tool_call_id='8f1d54a4-6915-4faa-90a6-e7a05df0f125'


In [ ]:
# --- [최종 답변 생성 단계] ---
# (이전 셀에서 messages 리스트에 ToolMessage가 추가되었음)

# 도구 실행 결과가 포함된 messages를 다시 llm에 전달하여 최종 답변 생성
final_response = llm_with_tools.invoke(messages)

print(f"최종 AI 답변: {final_response.content}")

최종 AI 답변: [{'type': 'text', 'text': '부산의 현재 시간은 오후 6시 58분 19초입니다.', 'extras': {'signature': 'CqkIAdHtim+q1agUoOXYjW0uNi/tcS5Na8UJv/VHyUmX8N0Uya2q9mE2X0y25ras1GzmViw6tTlekq3vb6dMEOvp/hF29TrVL7by92JCZXsjJXsnI5Q8VLnl61VcC/k2XEQ52ZXPPltP3glaJfAN0QUxJBCWTAoi3Fbp4I51i9uGJBtQE8GP0D4c5541UBjxyXCPdTXaPfwKU0GSsplVEVBfjKPR0VwCH6xIrQgc8NEAHokOElgIJU+9O1bvHhKs+IgKFLswmy+hMRevP2mj7+12UFnFEEX9gcnpvlCB7R39GbQ4gX+OYW4EoZUP5TLhjzH7/UBUvUCaaU2xVUqL6+5Ca3PDYDuW9UY8RO4INU+PZSF4xOA2IFO0Rh/CQi5cPIyFRhHnfa/odcRsAf9eWGfefYS7dnVewFxN4AVJLed7Yw7DyBn0+YZB/ToELRobtlE/arL6ed6VIyySzFzYGmxw1LFv6DYOoJUs8q53so8p18BdXczCJ5KcqV+vSTMe/qUdNLRI4Uitu9zspcLk7LNqJxBQvZO9qmqO1umQu73qa6qrWewE5eLk3mvJblTVVJryTCg8EF1d6o4p869aAaYRkAFPVVm+DdF1b4Wa47CS1QBkHh2tVlpdtvVo0yIOQf1MbWqYAoE8hEWn5Kb8GIRc7EcyivOTLrtVKRiQm6+8typz3/ecyOvFM/w/2SJ3mjQvOBoJt/jdNTlxH7G4mvflFD5qEm1zSMM7R59iHwrX6cbmwt43HqngQbXPRsTQ7jGBU2DoJVs9I50tnTluXbs7Zz1qJ1e7aXJtxwY8c/vy4LiyewJjUZdsNXh900GFp2ZC8TWhaaHrsRJr8bXONp2/QhYzPYkjvg2OhMJDi61e1VHU0WaztyjwGPdSiwbX11xY8rh80Wd

In [ ]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")


In [ ]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown() 

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)

In [ ]:
messages.append(HumanMessage("두산에너빌리티는 한달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content=[{'type': 'text', 'text': '두산에너빌리티의 티커(ticker)를 몰라서 검색으로 먼저 찾아볼게요. 잠시만 기다려주세요.', 'extras': {'signature': 'CsELAdHtim+uaPa+hkZAYzB69napeiCkDJd+Kkq/lgwWaQC+VI3yqnvKtTe4AQM0+wImgt+vvACOYbYd/sxgPcpBjCtM941YyAYMLpgkxSwxuJ0w5VgIDt/7CesbxPKZKjeuhz44lb5s0f4paoklwvO7XeMkaFcYQ6y4u0tCAggaBwZDVcLsMd5zTFVuhvAWM7+GvApAt6mWo/Ux1W9al9uqmsycKySexuuFexPv88JpgdbeuuCW4pSPsaK5Y7ZYBF8y/mMMxYKdjG1AIEIbODleiNV86XRWD1gf+kqMceCqDql60kPTEpntYc61SftPwk8US9vwLhVGRF1/5PQZI3kJUSKNU0O2miiL+CylqywGa5zyJv1xQ5tlZsa4RnGVVAQ93UrBh52RSQ8iP+QzizKaKmEq4vVmQpENyX5qGbpv46U+Zlc0L7MOgidU3YC5TMJcQuV8DHvnWXkr7K6AkbUfagytf5R4QfFIT8QqghdxTDaFkkC3Xe8+IHKcJ/K8u7wQfPM+gTikHUg52a9Lev40UekGq6LuvjuEg6nAUf35hfjzoaatKuSTs3WUHo99pY0/rPW9FYicaGYaA2rmrgSgu3qALsApEFWhPzdOkklfiHx5VgO7/V4l4XF/RAVknIb+iARy8GJXpYOwDQUCMzDWuSKMstGxw1xSoG7VLZOlXqwYqrki+H4bj7twCzkhOl1FfreP2UhrWikxFWVaF8ld+XDhb3XRlPR5pdmjDovT8IbDQAo8lYB0ZfGpThruRfVRuUSAcf39UBD/PgontT2EB3Eyan4CqW6N1sEKD+jR8qb/gTSvdpVJEk92T0OEBZOF59lLXQO7f9tmW9D0CFJUzshYz8FasoIRRNTlYpzJuZKeYaSB1B

In [ ]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

In [ ]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--05f6157e-6830-43da-908f-c50954465dcb-0', usage_metadata={'input_tokens': 383, 'output_tokens': 0, 'total_tokens': 383, 'input_token_details': {'cache_read': 0}})

In [ ]:
if 'GOOGLE_API_KEY' in os.environ:
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

    print("--- Gemini API 스트리밍 시작 ---")
    prompt = [HumanMessage("잘 지냈어? 한국 사회의 문제점에 대해 이야기해줘.")]
    
    # 💡 flush=True 추가
    for chunk in llm.stream(prompt):
        print(chunk.content, end='|', flush=True)
    
    print("\n--- 스트리밍 종료 ---")

--- Gemini API 스트리밍 시작 ---
네, 잘 지냈습니다! 한국 사회의 문제점에 대해 이야기해 드릴 수 있습니다. 한국은 지난 수십 년간 눈부신 경제 성장을 이루었지만, 그 과정에서 여러 사회적 문제점들이 나타나고 심|화되고 있습니다. 주요 문제점들을 몇 가지로 나누어 설명해 드릴게요.

1.  **초저출산 및 급격한 고령화:**
    *   **문제점:** 세계 최저 수준의 출|산율(2023년 합계출산율 0.72명)과 OECD 국가 중 가장 빠른 고령화 속도는 한국 사회의 지속 가능성을 위협하는 가장 심각한 문제입니다.
    *   **|원인:** 높은 양육비 및 사교육비 부담, 여성의 경력 단절 문제, 과도한 경쟁 사회, 불안정한 주거 문제, 일-가정 양립의 어려움 등이 복합적으로 작용합니다.|
    *   **영향:** 생산가능인구 감소, 미래 성장 동력 약화, 연금 및 복지 시스템의 재정 부담 증가, 국방력 약화 등으로 이어집니다.

2.  **경제|적 불평등 심화 및 양극화:**
    *   **문제점:** 소득 불평등과 자산 불평등이 심화되어 사회적 갈등의 주요 원인이 되고 있습니다. 특히 부동산 가격| 폭등은 자산 불평등을 더욱 심화시켰습니다.
    *   **원인:** 정규직-비정규직 간의 임금 격차, 대기업과 중소기업 간의 격차,| 자산 소득의 불균형, 고용 없는 성장 등이 복합적으로 작용합니다.
    *   **영향:** 청년층의 좌절감, 계층 이동의 어려움, 사회 통합 저해, 내|수 경제 위축 등으로 이어집니다.

3.  **과도한 경쟁 사회와 정신 건강 문제:**
    *   **문제점:** 입시 경쟁, 취업 경쟁, 직장 내 경쟁 등 사회 전반에| 걸친 과도한 경쟁은 국민들의 정신 건강을 위협하고 있습니다.
    *   **원인:** 학벌 위주 사회, 불안정한 고용 시장, 장시간 노동 문화, 성과주의 등이 경쟁을 부|추깁니다.
    *   **영향:** OECD 국가 중 가장 높은 자살률, 우울증 및 불안 장애 

In [ ]:
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.stream(messages)
is_first = True
for chunk in response:
    print("chunk type: ", type(chunk))
    if is_first:
        is_first = False
        gathered = chunk
    else:
        gathered += chunk

    print("content: ", gathered.content, "tool_call_chunk", gathered.tool_calls)

messages.append(gathered)



# (6) 생성된 llm 답변 출력
print(messages)

chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'location': '부산', 'timezone': 'Asia/Seoul'}, 'id': '161dd255-7b81-4f62-89a3-ce4d47ce21bb', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'location': '부산', 'timezone': 'Asia/Seoul'}, 'id': '161dd255-7b81-4f62-89a3-ce4d47ce21bb', 'type': 'tool_call'}]
[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessageChunk(content='', additional_kwargs={'function_call': {'name': 'get_current_time', 'arguments': '{"location": "\\ubd80\\uc0b0", "timezone": "Asia/Seoul"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_gena

In [ ]:
gathered

AIMessageChunk(content='', additional_kwargs={'function_call': {'name': 'get_current_time', 'arguments': '{"location": "\\ubd80\\uc0b0", "timezone": "Asia/Seoul"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--ffff8c37-9c86-453b-8a35-825f940e599b', tool_calls=[{'name': 'get_current_time', 'args': {'location': '부산', 'timezone': 'Asia/Seoul'}, 'id': '161dd255-7b81-4f62-89a3-ce4d47ce21bb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 261, 'output_tokens': 213, 'total_tokens': 474, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 189}}, tool_call_chunks=[{'name': 'get_current_time', 'args': '{"location": "\\ubd80\\uc0b0", "timezone": "Asia/Seoul"}', 'id': '161dd255-7b81-4f62-89a3-ce4d47ce21bb', 'index': None, 'type': 'tool_call_chunk'}], chunk_position='last')

In [ ]:
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # tool_dict를 사용하여 도구 이름으로 도구 함수를 선택
    print(tool_call["args"]) # 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'location': '부산', 'timezone': 'Asia/Seoul'}


AttributeError: 'function' object has no attribute 'invoke'